In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
basedir = "/Users/bono/Desktop/gm2FieldAnalysis/MuonConvolution/"

# Get the muon/ctag data

In [2]:
path = basedir + 'muon_info'
sys.path.append(path)
from database_helper import muon_formatter, time_to_run

In [3]:
df_ctag = muon_formatter(start='2018-04-22 00:00:00',end='2018-04-25 00:00:00')

In [4]:
df_ctag.head()

start_time            end_time  ctags quad_condition  \
run   subrun                                                                 
15921 0      2018-04-22 13:14:00 2018-04-22 13:14:34      0      12.1/18.0   
      5      2018-04-22 13:15:20 2018-04-22 13:15:29      0      13.1/18.0   
      11     2018-04-22 13:16:23 2018-04-22 13:16:31      0      13.1/18.0   
      17     2018-04-22 13:17:24 2018-04-22 13:17:28  20405      13.1/18.3   
      25     2018-04-22 13:18:24 2018-04-22 13:18:29  20205      13.1/18.3   

             kicker_condition  quad_ok  ctags_ok  losses_ok  fillcuts_ok  \
run   subrun                                                               
15921 0         3.6/2.64/2.76    False     False      False        False   
      5         3.6/2.64/2.76    False     False      False        False   
      11        3.6/2.64/2.76    False     False      False        False   
      17        3.6/2.64/2.76     True      True       True         True   
      25        3.6/2.64/2.76     True      True       True         True   

             field_ok  trolley_period  field_period ctags_loose_ok  \
run   subrun                                                         
15921 0          True             0.0           1.0          False   
      5          True             0.0           1.0          False   
      11         True             0.0           1.0          False   
      17         True             0.0           1.0           True   
      25         True             0.0           1.0           True   

             quad_loose_ok ctags_repeat_ok losses_repeat_ok  \
run   subrun                                                  
15921 0               None            None             None   
      5               None            None             None   
      11              None            None             None   
      17              None            None             None   
      25              None            None             None   

             fillcuts_repeat_ok           poor_time  poor_ctags  
run   subrun                                                     
15921 0                    None 2018-04-22 13:14:26       231.5  
      5                    None 2018-04-22 13:15:26         0.0  
      11                   None 2018-04-22 13:16:26         0.0  
      17                   None 2018-04-22 13:17:26       372.7  
      25                   None 2018-04-22 13:18:26       467.8

# Get the field data

In [5]:
path=basedir + 'field_info/'
sys.path.append(path)
from field_grid import *
from format_field import get_field_df

In [6]:
path=path + 'data/'
# file = "60Hr_Average_Comparison_all_3956_3997_norescut.txt"
file = "60Hr_Average_Comparison_all_3956_3997_norescut_NoAvg.txt"
file = path + file
df_field = get_field_df(file)
df_field.reset_index(inplace=True)
df_field.head()

,DateTime,D,eD,NQ,eNQ,SQ,eSQ,NS,eNS
0,2018-04-22 07:52:40,841.744,0.046244,0.060869,0.087027,0.163640,0.028346,-1.14625,0.028019
1,2018-04-22 07:52:42,841.765,0.046244,0.073984,0.087027,0.162312,0.028346,-1.14114,0.028019
2,2018-04-22 07:52:44,841.770,0.046244,0.045627,0.087027,0.161684,0.028346,-1.14125,0.028019
3,2018-04-22 07:52:46,841.767,0.046244,0.085345,0.087027,0.162837,0.028346,-1.14056,0.028019
4,2018-04-22 07:52:48,841.793,0.046244,0.058946,0.087027,0.164271,0.028346,-1.13227,0.028019


## Add a run/subrun column

In [21]:
times = df_field["DateTime"].tolist()
l = [time_to_run(t, df_ctag) for t in times]
df_field['run'] = [v[0][0] for v in l]
df_field['subrun'] = [v[0][1] for v in l]
df_field.set_index(['run','subrun'],inplace=True)
df_field.tail()

DateTime        D        eD        NQ       eNQ  \
run subrun                                                              
-1  -1     2018-04-25 07:20:22  840.834  0.046195  0.225935  0.087159   
    -1     2018-04-25 07:20:24  840.880  0.046195  0.216338  0.087159   
    -1     2018-04-25 07:20:26  840.803  0.046195  0.236885  0.087159   
    -1     2018-04-25 07:20:28  840.804  0.046195  0.219816  0.087159   
    -1     2018-04-25 07:20:30  840.844  0.046195  0.206743  0.087159   

                  SQ       eSQ       NS       eNS  
run subrun                                         
-1  -1      0.372770  0.028862 -1.27429  0.028667  
    -1      0.373393  0.028862 -1.25283  0.028667  
    -1      0.370688  0.028862 -1.27783  0.028667  
    -1      0.371314  0.028862 -1.27620  0.028667  
    -1      0.373385  0.028862 -1.26274  0.028667

## Do an inner join

In [22]:
df_total = pd.merge(df_field, df_ctag, on=['run','subrun'], how='inner')
df_total.head()

DateTime        D        eD        NQ       eNQ  \
run   subrun                                                              
15921 0      2018-04-22 13:14:02  841.735  0.079075  0.032789  0.096987   
      0      2018-04-22 13:14:04  841.706  0.079075  0.060667  0.096987   
      0      2018-04-22 13:14:06  841.685  0.079075  0.054613  0.096987   
      0      2018-04-22 13:14:08  841.714  0.079075  0.044995  0.096987   
      0      2018-04-22 13:14:10  841.708  0.079075  0.055440  0.096987   

                    SQ       eSQ       NS       eNS          start_time  ...  \
run   subrun                                                             ...   
15921 0       0.169870  0.030298 -1.15897  0.035181 2018-04-22 13:14:00  ...   
      0       0.175534  0.030298 -1.16613  0.035181 2018-04-22 13:14:00  ...   
      0       0.173558  0.030298 -1.17401  0.035181 2018-04-22 13:14:00  ...   
      0       0.173786  0.030298 -1.16228  0.035181 2018-04-22 13:14:00  ...   
      0       0.173864  0.030298 -1.16585  0.035181 2018-04-22 13:14:00  ...   

             field_ok  trolley_period field_period ctags_loose_ok  \
run   subrun                                                        
15921 0          True             0.0          1.0          False   
      0          True             0.0          1.0          False   
      0          True             0.0          1.0          False   
      0          True             0.0          1.0          False   
      0          True             0.0          1.0          False   

              quad_loose_ok  ctags_repeat_ok  losses_repeat_ok  \
run   subrun                                                     
15921 0                None             None              None   
      0                None             None              None   
      0                None             None              None   
      0                None             None              None   
      0                None             None              None   

              fillcuts_repeat_ok           poor_time  poor_ctags  
run   subrun                                                      
15921 0                     None 2018-04-22 13:14:26       231.5  
      0                     None 2018-04-22 13:14:26       231.5  
      0                     None 2018-04-22 13:14:26       231.5  
      0                     None 2018-04-22 13:14:26       231.5  
      0                     None 2018-04-22 13:14:26       231.5  

[5 rows x 28 columns]

### Get the ctag averaged dipole

In [23]:
D = (df_total['D']*df_total['ctags']).sum()/df_total['ctags'].sum()
D

841.413353362968

### Get the ctag averaged dipole using the "poor" ctags

In [24]:
D = (df_total['D']*df_total['poor_ctags']).sum()/df_total['poor_ctags'].sum()
D

841.4022791831616

# Do the same thing, but without any dqc cuts

In [26]:
from simple_ctags import get_ctags
data = get_ctags("2018-04-22 00:00:00", "2018-04-25 00:00:00",'localhost')
df_muons = pd.DataFrame.from_dict(data, orient="index",columns=['ctags'])
df_muons.index.name = 'DateTime'
df_muons.index = pd.to_datetime(df_muons.index)
df_muons = df_muons.sort_values(by=['DateTime'])

In [27]:
file = "60Hr_Average_Comparison_all_3956_3997_norescut.txt"
file = path + file
df_field_old = get_field_df(file)

In [30]:
#Get the muons/ctag time interval (shorter than the field's time interval)
muons_time_interval = (df_muons.index.values[1].astype('int64')
                      - df_muons.index.values[0].astype('int64'))//1e9

#upsample each dataframe to 1 second                      
df_muons = df_muons.resample('1S').ffill()
df_field_old = df_field_old.resample('1S').ffill()

#Join the dataframes
df_total_old = df_field_old.join(df_muons)
df_total_old = df_total_old.fillna(0)

#calculate ctags per second and add it as a column
factor = 1/float(muons_time_interval)
df_total_old['ctags_per_second'] = df_total_old['ctags'].astype(float)*factor
df_total_old.head()

,D,eD,NQ,eNQ,SQ,eSQ,NS,eNS,ctags,ctags_per_second
DateTime,,,,,,,,,,
2018-04-22 08:00:55,841.743,0.046244,0.055759,0.087027,0.162954,0.028346,-1.14626,0.028019,488.0,488.0
2018-04-22 08:00:56,841.743,0.046244,0.055759,0.087027,0.162954,0.028346,-1.14626,0.028019,488.0,488.0
2018-04-22 08:00:57,841.743,0.046244,0.055759,0.087027,0.162954,0.028346,-1.14626,0.028019,488.0,488.0
2018-04-22 08:00:58,841.743,0.046244,0.055759,0.087027,0.162954,0.028346,-1.14626,0.028019,488.0,488.0
2018-04-22 08:00:59,841.743,0.046244,0.055759,0.087027,0.162954,0.028346,-1.14626,0.028019,488.0,488.0


In [34]:
total_ctags = df_total_old["ctags_per_second"].sum()
D = (df_total_old["D"]*df_total_old["ctags_per_second"]).sum()/total_ctags
D

841.4022102960946

__Which is about the same dipole results as when poor_ctag was used in the full dataframe__

# Finally show how the full dataframe can be produced with a single line of code

In [41]:
path = basedir + 'synthesis'
sys.path.append(path)
from combine_data_products import field_ctag_dqm

In [37]:
df_test = field_ctag_dqm()

In [38]:
df_test.head()

DateTime        D        eD        NQ       eNQ  \
run   subrun                                                              
15921 0      2018-04-22 13:14:02  841.735  0.079075  0.032789  0.096987   
      0      2018-04-22 13:14:04  841.706  0.079075  0.060667  0.096987   
      0      2018-04-22 13:14:06  841.685  0.079075  0.054613  0.096987   
      0      2018-04-22 13:14:08  841.714  0.079075  0.044995  0.096987   
      0      2018-04-22 13:14:10  841.708  0.079075  0.055440  0.096987   

                    SQ       eSQ       NS       eNS          start_time  ...  \
run   subrun                                                             ...   
15921 0       0.169870  0.030298 -1.15897  0.035181 2018-04-22 13:14:00  ...   
      0       0.175534  0.030298 -1.16613  0.035181 2018-04-22 13:14:00  ...   
      0       0.173558  0.030298 -1.17401  0.035181 2018-04-22 13:14:00  ...   
      0       0.173786  0.030298 -1.16228  0.035181 2018-04-22 13:14:00  ...   
      0       0.173864  0.030298 -1.16585  0.035181 2018-04-22 13:14:00  ...   

             field_ok  trolley_period field_period ctags_loose_ok  \
run   subrun                                                        
15921 0          True             0.0          1.0          False   
      0          True             0.0          1.0          False   
      0          True             0.0          1.0          False   
      0          True             0.0          1.0          False   
      0          True             0.0          1.0          False   

              quad_loose_ok  ctags_repeat_ok  losses_repeat_ok  \
run   subrun                                                     
15921 0                None             None              None   
      0                None             None              None   
      0                None             None              None   
      0                None             None              None   
      0                None             None              None   

              fillcuts_repeat_ok           poor_time  poor_ctags  
run   subrun                                                      
15921 0                     None 2018-04-22 13:14:26       231.5  
      0                     None 2018-04-22 13:14:26       231.5  
      0                     None 2018-04-22 13:14:26       231.5  
      0                     None 2018-04-22 13:14:26       231.5  
      0                     None 2018-04-22 13:14:26       231.5  

[5 rows x 28 columns]

### Save the dataframe

In [39]:
df_test.to_pickle(basedir + 'synthesis/data/60H_field_ctag_dqm.pkl')

## can also get the poor-style df with a single line

In [3]:
path = basedir + 'synthesis'
sys.path.append(path)
from combine_data_products import field_ctag_dqm
from combine_data_products import field_poor_ctag

In [4]:
field_poor_ctag()

,D,eD,NQ,eNQ,SQ,eSQ,NS,eNS,ctags,ctags_per_second
DateTime,,,,,,,,,,
2018-04-22 07:52:40,841.744,0.046244,0.060869,0.087027,0.163640,0.028346,-1.14625,0.028019,479.0,7.983333
2018-04-22 07:52:41,841.744,0.046244,0.060869,0.087027,0.163640,0.028346,-1.14625,0.028019,479.0,7.983333
2018-04-22 07:52:42,841.765,0.046244,0.073984,0.087027,0.162312,0.028346,-1.14114,0.028019,479.0,7.983333
2018-04-22 07:52:43,841.765,0.046244,0.073984,0.087027,0.162312,0.028346,-1.14114,0.028019,479.0,7.983333
2018-04-22 07:52:44,841.770,0.046244,0.045627,0.087027,0.161684,0.028346,-1.14125,0.028019,479.0,7.983333
2018-04-22 07:52:45,841.770,0.046244,0.045627,0.087027,0.161684,0.028346,-1.14125,0.028019,479.0,7.983333
2018-04-22 07:52:46,841.767,0.046244,0.085345,0.087027,0.162837,0.028346,-1.14056,0.028019,479.0,7.983333
2018-04-22 07:52:47,841.767,0.046244,0.085345,0.087027,0.162837,0.028346,-1.14056,0.028019,479.0,7.983333
2018-04-22 07:52:48,841.793,0.046244,0.058946,0.087027,0.164271,0.028346,-1.13227,0.028019,479.0,7.983333
